# Grafo de Comunidad HA

Este notebook visualiza la comunidad de Horizons Architecture desde `comunidad.yaml`.

**Instrucciones:**
1. Sube `comunidad.yaml` a Colab (o monta Google Drive)
2. Ejecuta todas las celdas
3. Interactúa con el grafo generado

In [ ]:
# Instalar dependencias
!pip install pyyaml networkx pyvis -q

In [ ]:
import yaml
import networkx as nx
from pyvis.network import Network
from google.colab import files
import json

In [ ]:
# Opción 1: Subir archivo manualmente
# uploaded = files.upload()
# yaml_content = list(uploaded.values())[0].decode('utf-8')

# Opción 2: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ajusta la ruta a tu archivo
YAML_PATH = '/content/drive/MyDrive/path/to/comunidad.yaml'

# Opción 3: Pegar contenido directamente (para pruebas)
# Descomenta y pega el YAML aquí:
yaml_content = '''
# Pega aquí el contenido de comunidad.yaml para pruebas rápidas
'''

In [ ]:
def load_community(path=None, content=None):
    """Carga comunidad.yaml desde archivo o contenido."""
    if path:
        with open(path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)
    elif content and content.strip() and not content.strip().startswith('#'):
        return yaml.safe_load(content)
    else:
        raise ValueError("Proporciona path o content válido")

# Cargar datos
try:
    data = load_community(path=YAML_PATH)
    print(f"✅ Cargado desde archivo: {YAML_PATH}")
except:
    try:
        data = load_community(content=yaml_content)
        print("✅ Cargado desde contenido pegado")
    except:
        print("❌ Error: Sube comunidad.yaml o pega el contenido")
        data = None

if data:
    print(f"   Nodos: {len(data.get('nodes', []))}")
    print(f"   Edges: {len(data.get('edges', []))}")

In [ ]:
def build_graph(data):
    """Construye un grafo NetworkX desde los datos."""
    G = nx.DiGraph()
    
    # Agregar nodos
    for node in data.get('nodes', []):
        node_id = node['id']
        G.add_node(
            node_id,
            label=node.get('name', node_id),
            type=node.get('type', 'unknown'),
            **{k: v for k, v in node.items() if k not in ['id', 'name', 'type']}
        )
    
    # Agregar edges
    for edge in data.get('edges', []):
        G.add_edge(
            edge['from'],
            edge['to'],
            type=edge.get('type', 'related'),
            **{k: v for k, v in edge.items() if k not in ['from', 'to', 'type']}
        )
    
    return G

if data:
    G = build_graph(data)
    print(f"✅ Grafo construido: {G.number_of_nodes()} nodos, {G.number_of_edges()} edges")

In [ ]:
def visualize_graph(G, height='700px', width='100%'):
    """Visualiza el grafo con PyVis."""
    
    # Crear red PyVis
    net = Network(
        height=height,
        width=width,
        bgcolor='#ffffff',
        font_color='#333333',
        directed=True,
        notebook=True,
        cdn_resources='remote'
    )
    
    # Colores por tipo de nodo
    colors = {
        'person': '#4CAF50',      # Verde
        'org': '#2196F3',          # Azul
        'unknown': '#9E9E9E'       # Gris
    }
    
    # Tamaños por tipo
    sizes = {
        'person': 20,
        'org': 30,
        'unknown': 15
    }
    
    # Agregar nodos
    for node_id in G.nodes():
        node_data = G.nodes[node_id]
        node_type = node_data.get('type', 'unknown')
        
        # Tooltip con info
        title = f"<b>{node_data.get('label', node_id)}</b><br>"
        title += f"Type: {node_type}<br>"
        for k, v in node_data.items():
            if k not in ['label', 'type']:
                title += f"{k}: {v}<br>"
        
        net.add_node(
            node_id,
            label=node_data.get('label', node_id),
            color=colors.get(node_type, colors['unknown']),
            size=sizes.get(node_type, sizes['unknown']),
            title=title,
            shape='dot' if node_type == 'person' else 'box'
        )
    
    # Colores de edges por tipo
    edge_colors = {
        'leads': '#FF5722',        # Naranja
        'works_at': '#4CAF50',     # Verde
        'serves': '#2196F3',       # Azul
        'partner': '#9C27B0',      # Púrpura
        'prospect': '#FFC107',     # Amarillo
        'uses': '#607D8B',         # Gris azul
        'knows': '#E91E63',        # Rosa
    }
    
    # Agregar edges
    for source, target in G.edges():
        edge_data = G.edges[source, target]
        edge_type = edge_data.get('type', 'related')
        
        net.add_edge(
            source,
            target,
            color=edge_colors.get(edge_type, '#999999'),
            title=edge_type,
            arrows='to'
        )
    
    # Configurar física
    net.set_options('''
    {
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "centralGravity": 0.01,
          "springLength": 100,
          "springConstant": 0.08
        },
        "solver": "forceAtlas2Based",
        "stabilization": {
          "iterations": 150
        }
      },
      "interaction": {
        "hover": true,
        "navigationButtons": true
      }
    }
    ''')
    
    return net

if data:
    net = visualize_graph(G)
    net.show('grafo_comunidad.html')
    print("✅ Grafo generado: grafo_comunidad.html")

## Análisis del Grafo

In [ ]:
if data:
    print("📊 ANÁLISIS DEL GRAFO")
    print("=" * 40)
    
    # Conteo por tipo
    persons = [n for n, d in G.nodes(data=True) if d.get('type') == 'person']
    orgs = [n for n, d in G.nodes(data=True) if d.get('type') == 'org']
    
    print(f"\n👥 Personas: {len(persons)}")
    for p in persons:
        print(f"   - {G.nodes[p].get('label', p)}")
    
    print(f"\n🏢 Organizaciones: {len(orgs)}")
    for o in orgs:
        print(f"   - {G.nodes[o].get('label', o)}")
    
    # Nodos más conectados
    print(f"\n🔗 Nodos más conectados:")
    degrees = sorted(G.degree(), key=lambda x: x[1], reverse=True)[:5]
    for node, degree in degrees:
        print(f"   - {G.nodes[node].get('label', node)}: {degree} conexiones")
    
    # Tipos de relaciones
    print(f"\n🔀 Tipos de relaciones:")
    edge_types = {}
    for _, _, d in G.edges(data=True):
        t = d.get('type', 'unknown')
        edge_types[t] = edge_types.get(t, 0) + 1
    for t, count in sorted(edge_types.items(), key=lambda x: -x[1]):
        print(f"   - {t}: {count}")

## Exportar a otros formatos

In [ ]:
if data:
    # Exportar a JSON (para D3.js u otras visualizaciones)
    graph_json = nx.node_link_data(G)
    with open('grafo_comunidad.json', 'w') as f:
        json.dump(graph_json, f, indent=2, default=str)
    print("✅ Exportado: grafo_comunidad.json")
    
    # Exportar a GraphML (para Gephi)
    nx.write_graphml(G, 'grafo_comunidad.graphml')
    print("✅ Exportado: grafo_comunidad.graphml")

## Generar archivos .md para Obsidian (opcional)

In [ ]:
def generate_obsidian_files(data, output_dir='obsidian_output'):
    """Genera archivos .md para cada nodo."""
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Crear lookup de nodos
    nodes_lookup = {n['id']: n for n in data.get('nodes', [])}
    
    # Crear lookup de edges por nodo
    edges_by_node = {}
    for edge in data.get('edges', []):
        from_id = edge['from']
        to_id = edge['to']
        
        if from_id not in edges_by_node:
            edges_by_node[from_id] = {'outgoing': [], 'incoming': []}
        if to_id not in edges_by_node:
            edges_by_node[to_id] = {'outgoing': [], 'incoming': []}
        
        edges_by_node[from_id]['outgoing'].append(edge)
        edges_by_node[to_id]['incoming'].append(edge)
    
    # Generar .md por nodo
    for node in data.get('nodes', []):
        node_id = node['id']
        node_type = node.get('type', 'unknown')
        name = node.get('name', node_id)
        
        # Frontmatter YAML
        content = f"---\n"
        content += f"id: {node_id}\n"
        content += f"type: {node_type}\n"
        for k, v in node.items():
            if k not in ['id', 'type', 'name']:
                if isinstance(v, list):
                    content += f"{k}: {v}\n"
                else:
                    content += f"{k}: \"{v}\"\n"
        content += f"---\n\n"
        
        # Título
        content += f"# {name}\n\n"
        
        # Conexiones
        node_edges = edges_by_node.get(node_id, {'outgoing': [], 'incoming': []})
        
        if node_edges['outgoing']:
            content += f"## Conexiones salientes\n\n"
            for edge in node_edges['outgoing']:
                target = nodes_lookup.get(edge['to'], {}).get('name', edge['to'])
                content += f"- **{edge.get('type', 'related')}** → [[{edge['to']}|{target}]]\n"
            content += "\n"
        
        if node_edges['incoming']:
            content += f"## Conexiones entrantes\n\n"
            for edge in node_edges['incoming']:
                source = nodes_lookup.get(edge['from'], {}).get('name', edge['from'])
                content += f"- [[{edge['from']}|{source}]] → **{edge.get('type', 'related')}**\n"
            content += "\n"
        
        # Guardar archivo
        filepath = os.path.join(output_dir, f"{node_id}.md")
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
    
    print(f"✅ Generados {len(data.get('nodes', []))} archivos en {output_dir}/")

# Descomenta para generar:
# if data:
#     generate_obsidian_files(data)

---

## Instrucciones para actualizar

1. Edita `comunidad.yaml` en Obsidian
2. Sube el archivo actualizado a este Colab
3. Ejecuta todas las celdas
4. El grafo se actualiza automáticamente

---

*Generado por Horizons Architecture*